Bioinformatics Algorithms: genome assembly

To assemble a genome from its shotgun fragments (complete collection of all possible fragments, all the same length), do the following:<br>
. turn the k-mer collection (p137_data.txt) into an adjacency list by running the code 'De Bruijn algorithm'.<br>
. use the adjacency list (p147_data.txt) to make an ordered list of the k-mers by running the code 'Euler's theorem: path'.<br>
. use the ordered list (p125_data.txt) to reconstruct the DNA string by running the code 'Un-shotgun'.

 Shotgun<br>
 Input: An integer k and a string Text.<br>
 Output: All possible k-mers that are part of Text, k(Text) (the k-mers in alphabetical order).

In [ ]:
with open('p120_data.txt') as f:
    firstline = f.readline().split('\n')
    k = int(firstline[0])
    secondline = f.readline().split('\n')
    text = secondline[0]
output = []
for i in range(len(text)-k+1):
    output.append(text[i:i+k])
    output.sort()
print('\n'.join(output))

Un-shotgun<br>
Input: ordered k-mers (k_set) that overlap perfectly and shift one base to the right each time.<br>
Output: re-assembled shotgunned string.

In [ ]:
with open('p125_data.txt') as f:
    k_set = f.read().split('\n')
f.close()
output = ''
for i in range(len(k_set)-1):
    output = output + k_set[i][0]
output = output + k_set[-1]
print(output)

Construct overlap graph<br>
Input: A collection Patterns of k-mers.<br>
Output: List (dim N nodes) of lists. The overlap graph Overlap(Patterns), in the form of an adjacency list. 

In [ ]:
with open('p128_data.txt') as f:
    k_set = f.read().split('\n')
f.close()
output = []
for i in range(len(k_set)):
    sub_output = []
    compare_set = list(k_set)
    compare_set.remove(k_set[i])
    for j in range(len(compare_set)):
        if k_set[i][1:] == compare_set[j][:-1]:
            sub_output.append(compare_set[j])
    if len(sub_output) > 0:
        print(k_set[i] + ' -> ' + sub_output[0])

De Bruijn graph<br>
Input: An integer k and a string Text.<br>
Output: DeBruijnk(Text), in the form of an adjacency list (dictionary).

In [ ]:
import collections
with open('p132_data.txt') as f:
    k = int(f.readline())
    text = f.readline().strip('\n')
f.close()
nodes = []
bruijn = {}
for i in range(len(text)-k+2):
    node = text[i:i+k-1]
    nodes.append(node)
for j in range(len(nodes)-1):
    if nodes[j] in bruijn:
        bruijn[nodes[j]].append(nodes[j+1])
    else:
        bruijn[nodes[j]] = [nodes[j+1]]
o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
for key, value in o_bruijn.items():
    print(key + ' -> ' + ','.join(value))

De Bruijn algorithm<br>
Input: A collection of k-mers Patterns.<br>
Output: The adjacency list of the de Bruijn graph DeBruijn(Patterns).

In [ ]:
import collections
with open('p137_data.txt') as f:
    patterns = f.read().split('\n')
f.close()
nodes = []
bruijn = {}
k=len(patterns[0])
for j in range(len(patterns)):
    node = patterns[j][:k-1]
    if node in bruijn:
        bruijn[node].append(patterns[j][-(k-1):])
    else:
        bruijn[node] = [patterns[j][-(k-1):]]
o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
for key, value in o_bruijn.items():
    value.sort()
    print(key + ' -> ' + ','.join(value))

Euler's theorem: cycle<br>
Input: The adjacency list of an Eulerian directed graph.<br>
Output: An Eulerian cycle in this graph (6->8->7->9->6->5->4->2->1->0->3->2->6).

In [ ]:
import re
bruijn = {}
with open('p146_data.txt') as f:
    for line in f:
        numbers = re.findall('([0-9]+)',line)
        key = numbers[0]
        values = numbers[1:]
        for i in range(len(values)):
            v = values[i]
            values[i] = v
        bruijn[key] = values
f.close()
bruijn_leftover = bruijn
cycle = []
start = list(bruijn.keys())[0]
cycle.append(start)
cycle_next_step = bruijn[start][0]
current_position = start

#find the first cycle
while cycle_next_step != start:
    cycle.append(cycle_next_step)
    current_position = cycle_next_step
    cycle_next_step = bruijn[current_position][0]
    bruijn_leftover[cycle[-2]].remove(current_position)
bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
unexplored_edges = []
for key in bruijn_leftover:
    unexplored_edges.extend(bruijn_leftover[key])

#find the other cycles
while len(unexplored_edges) > 1:
    #find an unexplored edge in cycle
    for edge in cycle:
        if len(bruijn_leftover[edge]) > 0:
            new_start = edge
    #re-order cycle to start at new_start
    cycle = cycle[cycle.index(new_start):] + cycle[:cycle.index(new_start)]
    #unexplored_edges.remove(new_start)
    current_position = new_start
    unexplored_edges.remove(new_start)
    cycle.append(current_position)
    cycle_next_step = bruijn_leftover[current_position][0]
    while cycle_next_step != new_start:
        cycle.append(cycle_next_step)
        unexplored_edges.remove(cycle_next_step)
        current_position = cycle_next_step
        cycle_next_step = bruijn_leftover[current_position][0]
        bruijn_leftover[cycle[-2]].remove(current_position)
    bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
cycle.append(cycle_next_step)
#print(bruijn_leftover)
output = []
for j in range(len(cycle)):
    output.append(str(cycle[j]))
#print('->'.join(output))
print('\n'.join(output))

Euler's theorem: path<br>
Input: The adjacency list of a directed graph that has an Eulerian path.<br>
Output: An Eulerian path in this graph  6->7->8->9->6->3->0->2->1->3->4.<br>

In [ ]:
import re
bruijn = {}
with open('p147_data.txt') as f:
    for line in f:
        numbers = re.findall('([0-9A-Z]+)',line)
        key = numbers[0]
        values = numbers[1:]
        for i in range(len(values)):
            v = values[i]
            values[i] = v
        bruijn[key] = values
f.close()
cycle = []

#find the in- and outnodes; they are unbalanced, i.e., the number of indegrees != number of outdegrees
balance = {}
for key in bruijn:
    if key in balance:
        balance[key] = balance[key] + len(bruijn[key])
    else:
        balance[key] = len(bruijn[key])
    for j in range(len(bruijn[key])):
        if bruijn[key][j] in balance:
            balance[bruijn[key][j]] = balance[bruijn[key][j]] - 1
        else:
            balance[bruijn[key][j]] = -1
for key in balance:
    if balance[key] == 1: beginning = key
    if balance[key] == -1: end = key

#connect the unbalanced nodes
if end in bruijn:
    bruijn[end].append(beginning)
else:
    bruijn[end] = [beginning]
    
#find the first cycle
bruijn_leftover = bruijn
cycle = []
start = beginning
cycle.append(start)
cycle_next_step = bruijn[start][0]
current_position = start
while cycle_next_step != start:
    cycle.append(cycle_next_step)
    current_position = cycle_next_step
    cycle_next_step = bruijn[current_position][0]
    bruijn_leftover[cycle[-2]].remove(current_position)
bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
unexplored_edges = []
for key in bruijn_leftover:
    unexplored_edges.extend(bruijn_leftover[key])
    
#find the other cycles
while len(unexplored_edges) > 1:
    #find an unexplored edge in cycle
    for edge in cycle:
        if len(bruijn_leftover[edge]) > 0:
            new_start = edge
    #re-order cycle to start at new_start
    cycle = cycle[cycle.index(new_start):] + cycle[:cycle.index(new_start)]
    #unexplored_edges.remove(new_start)
    current_position = new_start
    unexplored_edges.remove(new_start)
    cycle.append(current_position)
    cycle_next_step = bruijn_leftover[current_position][0]
    while cycle_next_step != new_start:
        cycle.append(cycle_next_step)
        unexplored_edges.remove(cycle_next_step)
        current_position = cycle_next_step
        cycle_next_step = bruijn_leftover[current_position][0]
        bruijn_leftover[cycle[-2]].remove(current_position)
    bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
cycle.append(cycle_next_step)

#re-order cycle by cutting after end
if cycle[-1] != end:
    for c in range(len(cycle)-1):
        if cycle[c] == end and cycle[c+1] == beginning: cutting_point = c+1
    cycle = cycle[cutting_point:-1] + cycle[:cutting_point]

#print(bruijn_leftover)
output = []
for j in range(len(cycle)):
    output.append(str(cycle[j]))
print('\n'.join(output))

k-Universal binary circular string<br>
Input: An integer k.<br>
Output: A k-universal binary circular string.

In [ ]:
#produce all possible k-mers and proceed as described at the top
import itertools
k = 9
collection = list(itertools.product('01', repeat=k))
for i in range(len(collection)):
    print(''.join(collection[i]))

Solve the String Reconstruction from Read-Pairs Problem.<br>
Input: Integers k and d followed by a collection of paired k-mers PairedReads.<br>
Output: A string Text with (k, d)-mer composition equal to PairedReads.

In [ ]:
import re
first_patterns = []
second_patterns = []
with open('p149_data.txt') as f:
    parameters = f.readline().strip()
    k = int(re.findall('([0-9]+)',parameters)[0])
    d = int(re.findall('([0-9]+)',parameters)[1])
    for line in f:
        line.strip()
        numbers = re.findall('([0-9A-Z]+)',line)
        values = numbers
        for i in range(len(values)):
            v = values[i]
            values[i] = v
        first_patterns.append(values[0])
        second_patterns.append(values[1])
f.close()

graph = de_bruijn_algorithm_for_pairs(first_patterns,second_patterns)
ordered_kmers = eulers_theorem_path(graph)
genome = unshotgun_paired(ordered_kmers)
print(genome)

#CACCGATACTGATTCTGAAGCTT

In [ ]:
def de_bruijn_algorithm_for_pairs(kmers1,kmers2):
    import collections
    bruijn = {}
    k=len(kmers1[0])
    for j in range(len(kmers1)):
        key = kmers1[j][:-1] + kmers2[j][:-1]
        value = [kmers1[j][1:] + kmers2[j][1:]]
        if key in bruijn:
            bruijn[key].extend(value)
        else:
            bruijn[key] = value
    o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
    return o_bruijn

In [ ]:
#code p137 as a function
def de_bruijn_algorithm(kmers):
    import collections
    bruijn = {}
    k=len(kmers[0])
    for j in range(len(kmers)):
        node = kmers[j][:k-1]
        if node in bruijn:
            bruijn[node].append(kmers[j][-(k-1):])
        else:
            bruijn[node] = [kmers[j][-(k-1):]]
    o_bruijn = collections.OrderedDict(sorted(bruijn.items()))
    return o_bruijn

In [ ]:
#code p147 as a function
def eulers_theorem_path(ordered_dict):
    import re
    import collections
    bruijn = dict()
    for key, value in ordered_dict.items():
        value.sort()
        bruijn[key] = value
    cycle = []

    #find the in- and outnodes; they are unbalanced, i.e., the number of indegrees != number of outdegrees
    balance = {}
    for key in bruijn:
        if key in balance:
            balance[key] = balance[key] + len(bruijn[key])
        else:
            balance[key] = len(bruijn[key])
        for j in range(len(bruijn[key])):
            if bruijn[key][j] in balance:
                balance[bruijn[key][j]] = balance[bruijn[key][j]] - 1
            else:
                balance[bruijn[key][j]] = -1
    for key in balance:
        if balance[key] == 1: beginning = key
        if balance[key] == -1: end = key
    
    #connect the unbalanced nodes
    if end in bruijn:
        bruijn[end].append(beginning)
    else:
        bruijn[end] = [beginning]
    print('HELLO',beginning,end)
    
    #find the first cycle
    bruijn_leftover = bruijn
    cycle = []
    start = beginning
    cycle.append(start)
    cycle_next_step = bruijn[start][0]
    current_position = start
    while cycle_next_step != start:
        cycle.append(cycle_next_step)
        current_position = cycle_next_step
        cycle_next_step = bruijn[current_position][0]
        bruijn_leftover[cycle[-2]].remove(current_position)
    bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
    unexplored_edges = []
    for key in bruijn_leftover:
        unexplored_edges.extend(bruijn_leftover[key])

    #find the other cycles
    while len(unexplored_edges) > 1:
        #find an unexplored edge in cycle
        for edge in cycle:
            if len(bruijn_leftover[edge]) > 0:
                new_start = edge
        #re-order cycle to start at new_start
        cycle = cycle[cycle.index(new_start):] + cycle[:cycle.index(new_start)]
        #unexplored_edges.remove(new_start)
        current_position = new_start
        unexplored_edges.remove(new_start)
        cycle.append(current_position)
        cycle_next_step = bruijn_leftover[current_position][0]
        while cycle_next_step != new_start:
            cycle.append(cycle_next_step)
            unexplored_edges.remove(cycle_next_step)
            current_position = cycle_next_step
            cycle_next_step = bruijn_leftover[current_position][0]
            bruijn_leftover[cycle[-2]].remove(current_position)
        bruijn_leftover[cycle[-1]].remove(bruijn_leftover[cycle[-1]][0])
    cycle.append(cycle_next_step)

    #re-order cycle by cutting after end
    if cycle[-1] != end:
        for c in range(len(cycle)-1):
            if cycle[c] == end and cycle[c+1] == beginning: cutting_point = c+1
        cycle = cycle[cutting_point:-1] + cycle[:cutting_point]

    #print(bruijn_leftover)
    output = []
    for j in range(len(cycle)):
        output.append(str(cycle[j]))
    print('OUTPUT',output)
    return output

In [ ]:
#code p125 as a function
def unshotgun(ordered_kmers):
    output = ''
    for i in range(len(ordered_kmers)-1):
        output = output + ordered_kmers[i][0]
    output = output + ordered_kmers[-1]
    return output

All Eulerian cycles in a graph<br>
Input: a non-simple graph (as an OrderedDict)<br>
Output: a collection of Eulerian cycles

In [ ]:
def unshotgun_paired(ordered_pairs):
    print(ordered_pairs)
    prefix = ''
    suffix = ''
    #convert back to paired data
    paired_data = []
    prefix = ordered_pairs[0][:k-1]
    suffix = ordered_pairs[0][-k+1:]
    for i in range(len(ordered_pairs)-1):
        prefix = prefix + ordered_pairs[i+1][k-2]
        suffix = suffix + ordered_pairs[i+1][-1]
    print(prefix,suffix)
    for j in range(len(suffix)-(k+d)):
        print(prefix[-(j+1)] , suffix[-(k+d+j+1)])
    return prefix + suffix[-(k+d):]

MaximalNonBranchingPaths.<br>
Input: The adjacency list of a graph whose nodes are integers.<br>
Output: The collection of all maximal nonbranching paths in this graph.

In [ ]:
adjacency_dict = {1:[2],2:[3],3:[4,5],6:[7],7:[6]}
#def max_non_branching_paths(adj_list):
adjacency_dict_remaining = adjacency_dict
max_non_branching_paths = []
active_key = list(adjacency_dict_remaining.keys())[0]
while len(adjacency_dict_remaining) > 0:
    b_path = []
    if len(adjacency_dict_remaining[active_key]) == 1:
        b_path.append(active_key)
        b_path.append(adjacency_dict_remaining[active_key][0])
        print(b_path)
        if adjacency_dict_remaining[active_key][0] in adjacency_dict_remaining:
            active_key = adjacency_dict_remaining[active_key]
            print('ONE')
        else:
            del adjacency_dict_remaining[active_key]
            active_key = list(adjacency_dict_remaining.keys())[0]
            print('TWO')
    else:
        active_key = list(adjacency_dict_remaining.keys())[0]
    print(b_path)
    max_non_branching_paths.append(b_path)
print(max_non_branching_paths)

In [ ]:
kmers = []
with open('p150_data.txt') as f:
    for line in f:
        kmers.append(line.strip())
print(kmers)
o_bruijn=(de_bruijn_algorithm(kmers))
print(o_bruijn)